In [1]:
import time

import numpy as np
import pandas as pd
import sklearn.datasets, sklearn.metrics, sklearn.model_selection, sklearn.tree

import subprocess, sys

In [2]:
!cd ../ && make print-SOURCES

SOURCES = cpp/criterion.cpp cpp/factories.cpp cpp/gbt.cpp cpp/lltrees.cpp cpp/metrics.cpp cpp/node.cpp cpp/tree.cpp cpp/wrapper.cpp


In [3]:
# !cd ../ && make clean
!cd ../ && make
subprocess.call([sys.executable, "lltrees_python_import_for_debug.py"])

make: Nothing to be done for 'all'.
[[ 0.43693817 -0.30246969  0.17041622 -0.38817682 -0.61340266  1.19064627
   0.16057398  0.00304602]
 [ 0.39579885  0.3594758  -0.71604344  2.34907579 -0.47875857  0.26355081
  -0.41662531 -0.06312301]
 [-0.53145455  0.31313214  1.46654086  0.6870658   1.37649645 -1.56785943
  -1.11158025 -0.03582558]]
Type of Training Data : float64
Configuration mode : regression
No Validate Data,  will use Training Data.
0 0 -35.4794 700 700 700
mae::get 97.7474 175.878 54154.9
mae::get 97.7474 175.878 54154.9
-49.0022 -49.0022 -35.4794 700 700 700
-49.0022 -49.0022 13.5227 700 700 700
Epoch :     1 Metric Train : 77.3641 Metric va : 77.3641 Residuals (mean) : -1.8531e-11
FIT --- 0.04820871353149414 seconds ---
PREDICT --- 2.765655517578125e-05 seconds ---
[-1.85309545e-11]
rmse: 96.17
mae: 76.64
r2: 0.40
Type of Training Data : float64
Configuration mode : regression
No Validate Data,  will use Training Data.
0 0 13.5227 700 700 700
mae::get -53.7383 78.1308 4412

0

# make_regression

In [4]:
X, Y = sklearn.datasets.make_regression(n_samples=1000, n_features=8, n_informative=5, n_targets=1, noise=1, random_state=42)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

In [5]:
sys.path.append('/home/alexandre/Desktop/lltrees/src/build')
import lltrees
conf ={
    'epochs' : 1,
    'learning_rate' : 1,
    'metric' : 'mae', # Possible metrics for the moment : mae, mse
    'max_depth' : 5,
    'min_leaf_size' : 2,
    'criterion' : "absolute_error",  # Possible metrics for the moment : variance, absolute_error
    'verbose' : 0, 
}
my_lltree = lltrees.lltree()
my_lltree.set_conf(conf)
my_lltree.get_conf()

start_time = time.time()
my_lltree.fit(X_train, Y_train, X_test, Y_test)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

ModuleNotFoundError: No module named 'lltrees'

In [ ]:
my_lltree = sklearn.tree.DecisionTreeRegressor(max_depth = 5, criterion ="absolute_error", random_state = 0)

start_time = time.time()
my_lltree.fit(X_train, Y_train)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print(YP[0:10])
print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

FIT --- 0.025666236877441406 seconds ---
PREDICT --- 0.00044417381286621094 seconds ---
[  -5.25635722  127.29805997 -110.4560012   122.24421396 -110.4560012
  -39.90414254  106.73581898  -39.90414254  223.68925908  240.37517626]
rmse: 63.68
mae: 49.35
r2: 0.74


In [ ]:
conf ={
    'epochs' : 50,
    'learning_rate' : 0.1,
    'metric' : 'mae', # Possible metrics for the moment : mae, mse
    'max_depth' : 5,
    'min_leaf_size' : 2,
    'criterion' : "absolute_error",  # Possible metrics for the moment : variance, absolute_error
    'verbose' : 0, 
}
my_lltree = lltrees.lltree()
my_lltree.set_conf(conf)
my_lltree.get_conf()

start_time = time.time()
my_lltree.fit(X_train, Y_train)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

FIT --- 3.09287428855896 seconds ---
PREDICT --- 0.0004191398620605469 seconds ---
rmse: 123.99
mae: 101.51
r2: -0.00
-----------------------------------------
mode :              regression
epochs :            50
learning_rate :     0
metric :            mae
criterion :         absolute_error
max_depth :         5
min_leaf_size :     2
verbose :           0
-----------------------------------------
Type of Training Data : float64
Configuration mode : regression
No Validate Data, will use Training Data.
0 0 -35.4794 700 700 700
-119.058
-66.3371
-72.0952
0.688991
-42.911
-71.9097
-38.7297
-130.19
1.88323
-1.72046
129.138
-14.3811
110.278
76.5077
25.0161
57.4411
67.6968
58.2411
123.421
mae::get 0 175.878 66593.7
mae::get 0 175.878 66593.7
0 0 -35.4794 700 700 700
0 0 -35.4794 700 700 700
-45.802
-61.2591
-128.181
-47.0886
-71.9097
-38.7297
-130.19
1.88323
-1.72046
64.078
61.0629
111.106
30.1322
76.0072
142.43
149.404
mae::get 0 175.878 66593.7
mae::get 0 175.878 66593.7
0 0 -35.4794 700

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
my_lltree = GradientBoostingRegressor(learning_rate = 0.1, n_estimators =50, 
                                 loss = 'absolute_error', criterion = 'squared_error',
                                 random_state = 0)

start_time = time.time()
my_lltree.fit(X_train, Y_train)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

FIT --- 0.14905500411987305 seconds ---
PREDICT --- 0.0008254051208496094 seconds ---
rmse: 44.56
mae: 31.80
r2: 0.87


# make_classification

In [ ]:
X, Y = sklearn.datasets.make_classification(n_samples=1000, n_features=8, n_informative=5, n_classes=2, random_state=42)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
# conf ={
#     'epochs' : 50,
#     'learning_rate' : 0.1,
#     'metric_name' : 'mae', # Possible metrics for the moment : mae, mse
#     'lltree_max_depth' : 3,
#     'lltree_min_size_split' : 1,
#     'lltree_criterion' : "gini",  # Possible metrics for the moment : variance, absolute_error
#     'verbose' : 1,  # Possible metrics for the moment : variance, absolute_error
# }
# my_lltree = lltrees.lltree()
# my_lltree.set_conf(conf)
# my_lltree.get_conf()

# start_time = time.time()
# my_lltree.fit(X_train, Y_train)
# print("FIT --- %s seconds ---" % (time.time() - start_time))

# start_time = time.time()
# YP = my_lltree.predict(X_test)
# print("PREDICT --- %s seconds ---" % (time.time() - start_time))

# print("accuracy_score: %.2f" % np.sqrt(sklearn.metrics.accuracy_score(Y_test,YP)))
# print("log_loss: %.2f" % sklearn.metrics.log_loss(Y_test,YP))
# print("f1_score: %.2f" % sklearn.metrics.f1_score(Y_test,YP))